In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

In [ ]:
# --- Configuración ---
USUARIO = "omar.tuesta@unilever.com"
CONTRASENA = "Ponds2025."
URL_LOGIN = "https://b2b.intercorpretail.pe/Finanzas/BBRe-commerce/main"


In [ ]:


# Cambia aquí el nombre si tu archivo tiene otro nombre o extensión
nombre_archivo = "BASE INRETAIL 2.XLSX"

# Busca el archivo en la misma carpeta del script
ruta_actual = os.path.dirname(os.path.abspath(__file__))
ruta_excel = os.path.join(ruta_actual, nombre_archivo)


def obtener_valores_columna_h(ruta_archivo):
    df = pd.read_excel(ruta_archivo, sheet_name="BD", engine="openpyxl")
    columna_h = df.iloc[1:, 7].astype(str)
    valores_limpios = columna_h.apply(lambda x: x.replace('.', ''))
    valores_numericos = [valor for valor in valores_limpios if valor.isdigit()]
    valores_unicos = sorted(set(valores_numericos))
    return valores_unicos

def iniciar_sesion(driver):
    driver.get(URL_LOGIN)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(USUARIO)
        driver.find_element(By.ID, "password").send_keys(CONTRASENA)
        driver.find_element(By.ID, "kc-login").click()
        print("✅ Inicio de sesión realizado")
    except Exception as e:
        print("❌ Error al iniciar sesión:", e)

def abrir_modulo_conciliaciones(driver):
    try:
        WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[@class='v-menubar-menuitem' and .//span[text()='Módulo Conciliaciones']]"))
        ).click()
        print("✅ Módulo Conciliaciones abierto")
    except Exception as e:
        print("❌ Error al abrir módulo de conciliaciones:", e)

def cambiar_ventana(driver, ventana_principal):
    time.sleep(2)
    for ventana in driver.window_handles:
        if ventana != ventana_principal:
            driver.switch_to.window(ventana)
            print("✅ Cambiado a nueva ventana:", driver.current_url)
            return
    print("❌ No se encontró nueva ventana")

def abrir_transacciones(driver):
    try:
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[@class='v-menubar-menuitem-caption' and text()='Transacciones']"))
        ).click()
        print("✅ Se abrió Transacciones")
    except Exception as e:
        print("❌ Error al abrir Transacciones:", e)

def seleccionar_opcion_por_contexto(driver, texto_contexto, texto_opcion):
    try:
        contenedores = WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.XPATH, f"//*[contains(text(), '{texto_contexto}')]"))
        )
        for contenedor in contenedores:
            try:
                dropdown = contenedor.find_element(By.XPATH, ".//following::div[contains(@class, 'v-filterselect-button')]")
                driver.execute_script("arguments[0].scrollIntoView(true);", dropdown)
                dropdown.click()
                time.sleep(2)
                opciones = WebDriverWait(driver, 20).until(
                    EC.presence_of_all_elements_located((By.XPATH, f"//*[contains(text(), '{texto_opcion}')]"))
                )
                for opcion in opciones:
                    if texto_opcion in opcion.text:
                        opcion.click()
                        print(f"✅ Opción seleccionada: {opcion.text}")
                        return
                print(f"❌ No se encontró la opción: {texto_opcion}")
                return
            except Exception:
                continue
        print(f"❌ No se encontró dropdown relacionado con: {texto_contexto}")
    except Exception as e:
        print(f"❌ Error al seleccionar opción '{texto_opcion}' con contexto '{texto_contexto}':", e)

def ingresar_valor_referencia(driver, valor):
    try:
        campo = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "v-textfield-bbr-filter-fields"))
        )
        campo.clear()
        campo.send_keys(valor)
        print(f"✅ Valor '{valor}' ingresado en el campo de referencia")
    except Exception as e:
        print(f"❌ Error al ingresar el valor en el campo de referencia:", e)
    try:
        Generar_informe = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'v-button') and .//span[text()='Generar Informe']]"))
        )
        Generar_informe.click()
        print("✅ Informe generado")
    except Exception as e:
        print(f"❌ Error al hacer clic en 'Generar Informe':", e)


def OC(driver):
    try:
                # 1. Clic en el botón con número de referencia
        boton_referencia = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@role='button' and contains(@class, 'grid-link-button-blue')]"))
        )
        texto_boton = boton_referencia.text
        boton_referencia.click()
        print(f"✅ Se hizo clic en el botón con número de referencia: {texto_boton}")
        time.sleep(4)

        # Buscar el botón de descarga dentro del popup de Detalle Transacción   
    
        boton_descarga = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "div.v-button.v-widget.toolbar-button.v-button-toolbar-button"))
        )
        boton_descarga.click()
        print("✅ Botón clickeado exitosamente")

        # 4. Clic en el label "Archivo Excel (xls)"
        label_excel = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//label[contains(text(), 'Archivo Excel (xls)')]"))
        )
        label_excel.click()
        print("✅ Se hizo clic en el label 'Archivo Excel (xls)'")
        time.sleep(1)

        # 5. Clic en el botón "Seleccionar"
        boton_seleccionar = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@role='button' and contains(@class, 'btn-generic')]//span[text()='Seleccionar']/ancestor::div[@role='button']"))
        )
        boton_seleccionar.click()
        print("✅ Se hizo clic en el botón 'Seleccionar'")
        time.sleep(2)

        # 6. Clic en el enlace de descarga
        enlace_descarga = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'Recepción_de_Mercadería')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", enlace_descarga)
        enlace_descarga.click()
        print("✅ Se hizo clic en el enlace de descarga del archivo Excel")
        time.sleep(7)

    except Exception as e:

        import traceback
        print("❌ Error en la función OC:", e)
        traceback.print_exc()


# --- Ejecución principal ---
if __name__ == "__main__":
    ruta_excel = "BASE INRETAIL 2.XLSX"
    valores_referencia = obtener_valores_columna_h(ruta_excel)
    print("Valores únicos de la columna H (solo números):")
    print(valores_referencia)

    driver = webdriver.Edge()  # O usa webdriver.Chrome() si prefieres Chrome
    iniciar_sesion(driver)
    ventana_principal = driver.current_window_handle

    for valor in valores_referencia:
        print(f"\n--- Procesando referencia: {valor} ---")
        
        # Desde aquí comienza el ciclo completo por cada valor
        abrir_modulo_conciliaciones(driver)
        cambiar_ventana(driver, ventana_principal)
        abrir_transacciones(driver)
        seleccionar_opcion_por_contexto(driver, "Sociedad", "INRETAIL PHARMA S.A.")
        seleccionar_opcion_por_contexto(driver, "Criterio", "No. Referencia")
        ingresar_valor_referencia(driver, valor)
        OC(driver)  # Incluye clic en botón de referencia, descarga y selección de archivo

    input("Presiona Enter para cerrar el navegador...")
    driver.quit()








